In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
import numpy as np
from random import shuffle
import tensorflow as tf
from sklearn import datasets, utils
from sklearn.preprocessing import label_binarize

In [52]:
iris = load_iris()
data_iris = np.column_stack([iris['data'], iris['target']])
np.random.shuffle(data_iris)
# print(data_iris)
target_multiclass = label_binarize(data_iris[:, -1], classes=[0, 1, 2])

# параметры
learning_rate = 0.001
training_epochs = 30
batch_size = 20
display_step = 1

#  параметры с
n_hidden_1 = 25 # первый слой
n_hidden_2 = 20 
n_input = 4 
n_classes = 3 

# tf Graph input
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


# Create model
def multilayer_perceptron(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Construct model
logits = multilayer_perceptron(X)

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# Initializing the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0
        total_batch = int(100/batch_size)
        train_x = data_iris[:100, :4]
        train_y = target_multiclass[:100]
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = train_x[i:i+batch_size], train_y[i:i+batch_size]
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([train_op, loss_op], feed_dict={X: batch_x,
                                                            Y: batch_y})
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost={:.9f}".format(avg_cost))
    print("Optimization Finished!")

# Test model
    pred = tf.nn.softmax(logits)  # Apply softmax to logits
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({X: data_iris[100:150,:4], Y: target_multiclass[100:150]}))

Epoch: 0001 cost=77.524002075
Epoch: 0002 cost=61.099717712
Epoch: 0003 cost=45.041584015
Epoch: 0004 cost=33.648373413
Epoch: 0005 cost=26.954037476
Epoch: 0006 cost=21.919200134
Epoch: 0007 cost=17.604640770
Epoch: 0008 cost=13.435563469
Epoch: 0009 cost=9.437077332
Epoch: 0010 cost=6.217611790
Epoch: 0011 cost=3.047055340
Epoch: 0012 cost=0.897659647
Epoch: 0013 cost=1.447161150
Epoch: 0014 cost=0.975597227
Epoch: 0015 cost=0.384572309
Epoch: 0016 cost=0.556627715
Epoch: 0017 cost=0.525082785
Epoch: 0018 cost=0.387204802
Epoch: 0019 cost=0.387488851
Epoch: 0020 cost=0.334774730
Epoch: 0021 cost=0.302031642
Epoch: 0022 cost=0.309449667
Epoch: 0023 cost=0.290446568
Epoch: 0024 cost=0.273943275
Epoch: 0025 cost=0.253701466
Epoch: 0026 cost=0.241063535
Epoch: 0027 cost=0.232649729
Epoch: 0028 cost=0.220446235
Epoch: 0029 cost=0.207250977
Epoch: 0030 cost=0.195967731
Optimization Finished!
Accuracy: 0.9
